# 正規化層

入力されたデータを正規化して返す層。

入力されたデータが特定の統計量（平均・分散）に従うように正規化される。  
統計量は学習可能なパラメータである。

平均0, 分散1への正規化+シフトとも見られる。

統計量を求める軸によって色々な種類がある。
- バッチ正規化
- 層正規化
- インスタンス正規化

In [1]:
import torch
from torch import nn


---

## バッチ正規化

*Batch Normalization*

バッチ方向に正規化を行う。  
学習時と推論時で挙動が変わる。

### 学習

学習時の挙動を見てみよう。  
まず適当なデータを用意する。

In [2]:
batch_size = 2
d = 5
x = torch.randn(batch_size, d)
x

tensor([[ 0.3220,  1.1343,  0.0410, -0.1697, -0.7343],
        [ 0.4345,  1.6453,  0.5363,  0.7546, -1.0348]])

バッチ正規化層を作る。pytorchを使う。

In [3]:
norm = nn.BatchNorm1d(d)
norm.train();

データを突っ込んでみよう。

In [4]:
y = norm(x)
y

tensor([[-0.9984, -0.9999, -0.9999, -1.0000,  0.9998],
        [ 0.9984,  0.9999,  0.9999,  1.0000, -0.9998]],
       grad_fn=<NativeBatchNormBackward0>)

平均0, 分散1に正規化された。  
これはpytorchが設定した初期値であって、学習によって変化する。

In [5]:
for params in norm.parameters():
    print(params)

Parameter containing:
tensor([1., 1., 1., 1., 1.], requires_grad=True)
Parameter containing:
tensor([0., 0., 0., 0., 0.], requires_grad=True)


学習時はミニバッチ前提なので、例えばバッチサイズ1のデータを入れるとエラーが出る。

In [6]:
x = torch.randn(1, d)
try:
    y = norm(x)
except Exception as e:
    print(e)

Expected more than 1 value per channel when training, got input size torch.Size([1, 5])


### 推論

推論時は挙動が変わる。

学習時と同じようにしてしまうと、バッチ内のデータによって結果が変わってしまう。サンプリング方法によって推論結果が変わるのはマズイ。  
後は、バッチサイズ1の時も困る。

推論時は、学習データ全体の統計量で正規化する。  
学習時に、特徴量ごとの統計量を記録しておく。

みてみよう。

In [7]:
batch_size = 2
d = 5
x = torch.randn(batch_size, d)
x

tensor([[ 1.3698, -1.6256,  0.7164, -1.3545,  0.4194],
        [ 0.1034, -0.5076,  1.0711, -0.6299, -0.3765]])

In [8]:
norm = nn.BatchNorm1d(d)
norm.eval();

In [9]:
y = norm(x)
y

tensor([[ 1.3698, -1.6256,  0.7164, -1.3545,  0.4194],
        [ 0.1034, -0.5076,  1.0711, -0.6299, -0.3765]],
       grad_fn=<NativeBatchNormBackward0>)

同じデータが出力された。  
これは、パラメータに加え、学習データ全体の統計量も平均0, 分散1として初期化されているから。

In [10]:
norm.state_dict()

OrderedDict([('weight', tensor([1., 1., 1., 1., 1.])),
             ('bias', tensor([0., 0., 0., 0., 0.])),
             ('running_mean', tensor([0., 0., 0., 0., 0.])),
             ('running_var', tensor([1., 1., 1., 1., 1.])),
             ('num_batches_tracked', tensor(0))])

`running_mean`と`running_var`が学習データ全体の統計量。  
これらは学習モードで演算を行うと更新される。

In [11]:
norm.train()
y = norm(x)
norm.state_dict()

OrderedDict([('weight', tensor([1., 1., 1., 1., 1.])),
             ('bias', tensor([0., 0., 0., 0., 0.])),
             ('running_mean',
              tensor([ 0.0737, -0.1067,  0.0894, -0.0992,  0.0021])),
             ('running_var', tensor([0.9802, 0.9625, 0.9063, 0.9263, 0.9317])),
             ('num_batches_tracked', tensor(1))])

In [12]:
print('mean:', x.mean(dim=0))
print('var:', x.var(dim=0))

mean: tensor([ 0.7366, -1.0666,  0.8937, -0.9922,  0.0214])
var: tensor([0.8019, 0.6250, 0.0629, 0.2625, 0.3168])


入力したデータの平均と分散が記録された。  
`momentum=0.1`がデフォルトで設定されているので、完全には一致していない。

これでもう一度推論モードにしてみると、今記録した統計量で演算が行われる。

In [13]:
norm.eval()
y = norm(x)
y

tensor([[ 1.3092, -1.5483,  0.6586, -1.3043,  0.4323],
        [ 0.0301, -0.4087,  1.0312, -0.5514, -0.3923]],
       grad_fn=<NativeBatchNormBackward0>)

確かめてみよう。

In [14]:
mean = norm.state_dict()['bias'] # 平均（パラメータ）
var = norm.state_dict()['weight'] # 分散（パラメータ）
data_mean = norm.state_dict()['running_mean'] # 平均（学習データ）
data_var = norm.state_dict()['running_var'] # 分散（学習データ）

((x - data_mean) / torch.sqrt(data_var + norm.eps)) * var + mean

tensor([[ 1.3092, -1.5483,  0.6586, -1.3043,  0.4323],
        [ 0.0301, -0.4087,  1.0312, -0.5514, -0.3923]])

ちゃんと同じになったね。

あと、バッチサイズ1の時もちゃんと動く。

In [15]:
x = torch.randn(1, d)
norm.eval()
norm(x)

tensor([[ 1.2279, -1.2460,  0.3038, -0.7483, -1.1732]],
       grad_fn=<NativeBatchNormBackward0>)